In [1]:
import pandas as pd

In [2]:
file_endings = "GSE243529"
normal_count = 128

In [3]:
train = pd.read_csv(f"../champ_result/GSE243529_train/all_beta_normalized_train.csv")

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
# id_list = pd.read_csv("../result/id_list.txt", header=None)

In [ ]:
# train = train[train["Unnamed: 0"].isin(id_list[0])]

In [ ]:
train

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,515,516,517,518,519,520,521,522,523,524
0,cg07881041,0.891637,0.891637,0.932066,0.932066,0.940202,0.940202,0.945641,0.945641,0.940250,...,0.927144,0.927144,0.954413,0.954413,0.960868,0.960868,0.943230,0.943230,0.946855,0.946855
1,cg03513874,0.942312,0.942312,0.935187,0.935187,0.964009,0.964009,0.966716,0.966716,0.945237,...,0.949742,0.949742,0.942481,0.942481,0.964317,0.964317,0.945584,0.945584,0.962746,0.962746
2,cg05451842,0.029975,0.029975,0.022880,0.022880,0.017531,0.017531,0.026281,0.026281,0.034589,...,0.030706,0.030706,0.025217,0.025217,0.019059,0.019059,0.031791,0.031791,0.017954,0.017954
3,cg14797042,0.983277,0.983277,0.989621,0.989621,0.987711,0.987711,0.960712,0.960712,0.966604,...,0.973674,0.973674,0.971978,0.971978,0.968671,0.968671,0.966904,0.966904,0.972501,0.972501
4,cg09838562,0.009447,0.009447,0.009020,0.009020,0.007164,0.007164,0.007786,0.007786,0.024895,...,0.009642,0.009642,0.012426,0.012426,0.020793,0.020793,0.014402,0.014402,0.018484,0.018484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736420,cg19812938,0.867959,0.867959,0.903661,0.903661,0.890041,0.890041,0.896655,0.896655,0.859622,...,0.878742,0.878742,0.899942,0.899942,0.897113,0.897113,0.885093,0.885093,0.878564,0.878564
736421,cg06272054,0.008151,0.008151,0.011305,0.011305,0.011354,0.011354,0.009927,0.009927,0.008303,...,0.016021,0.016021,0.012377,0.012377,0.011960,0.011960,0.006381,0.006381,0.014298,0.014298
736422,cg07255356,0.017649,0.017649,0.016949,0.016949,0.022004,0.022004,0.019642,0.019642,0.027163,...,0.020522,0.020522,0.013245,0.013245,0.021643,0.021643,0.024515,0.024515,0.029187,0.029187
736423,cg24220897,0.932095,0.932095,0.921312,0.921312,0.946663,0.946663,0.965407,0.965407,0.949585,...,0.923381,0.923381,0.861654,0.861654,0.904196,0.904196,0.929799,0.929799,0.953917,0.953917


In [ ]:
train_normal = train.iloc[:, 1:normal_count+1:2].T
train_tumor = train.iloc[:,normal_count+1::2].T

##### Calculate Δβ

In [ ]:
# remove outlier based on every column
def IQR(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    upper_fence = Q3 + IQR*1.5
    lower_fence = Q1 - IQR*1.5
    return upper_fence,lower_fence
def no_outlier(df):
    upper_fence, lower_fence = IQR(df)
    ddf=df[(df>lower_fence)&(df<upper_fence)]
    return ddf

In [ ]:
train_normal = no_outlier(train_normal)
train_tumor = no_outlier(train_tumor)

In [ ]:
train_normal_avg = train_normal.mean(skipna=True, axis = 0)

In [ ]:
train_tumor=(train_tumor).subtract(train_normal_avg, axis = 1)

In [ ]:
train_tumor=no_outlier(train_tumor)

In [ ]:
train_tumor_mean = train_tumor.mean(skipna=True, axis = 0)

In [ ]:
Δβ = pd.merge(train.iloc[:,:1], pd.DataFrame(train_tumor_mean, columns = ["dbeta"]), left_index=True, right_index=True)
Δβ

,Unnamed: 0,dbeta
0,cg07881041,-0.000410
1,cg03513874,0.000372
2,cg05451842,0.001294
3,cg14797042,-0.002949
4,cg09838562,0.001862
...,...,...
736420,cg19812938,-0.005165
736421,cg06272054,0.001076
736422,cg07255356,0.002739
736423,cg24220897,-0.005422


In [ ]:
# dmp_train = pd.read_csv(f"../champ_result/DMP_result_{file_endings}.csv")
dmp_train = pd.read_csv(f"../champ_result/GSE243529_train/DMP_result_train.csv")
print(f"raw train shape: {dmp_train.shape}")
dmp_train = dmp_train[["Unnamed: 0", "gene"]]
dmp_train.dropna(inplace=True)
print(f"after dropna: {dmp_train.shape}")

raw train shape: (162431, 24)
after dropna: (124270, 2)


In [ ]:
result = pd.merge(Δβ, dmp_train, on="Unnamed: 0", how="inner")

In [ ]:
def find_max_dBeta_grouped(group):
    idx_max = group['dbeta'].abs().idxmax()
    return group.loc[idx_max]

result_max_per_gene = result.groupby("gene").apply(find_max_dBeta_grouped).reset_index(drop=True)

C:\Users\Vincent\AppData\Local\Temp\ipykernel_13536\2529763667.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_max_per_gene = result.groupby("gene").apply(find_max_dBeta_grouped).reset_index(drop=True)


In [ ]:
single = pd.read_csv('../comorbidity/matchgene174_single_3Y10__OR2.txt', sep='\t', header=None)

In [ ]:
result_max_per_gene_single = result_max_per_gene[result_max_per_gene['gene'].isin(single[0])]
result_max_per_gene_single

,Unnamed: 0,dbeta,gene
1,cg16531903,0.005440,A1CF
4,cg00134295,0.011180,A2M
5,cg23546356,0.024950,A2ML1
12,cg13001012,0.009886,AADAC
16,cg00309402,0.011102,AADAT
...,...,...,...
18035,cg06155771,0.016239,ZNRD1
18038,cg18720930,0.008627,ZNRF3
18055,cg18599533,0.019920,ZSCAN18
18074,cg02286335,-0.010592,ZWINT


In [ ]:
result_max_per_gene_single.to_csv(f"../result/result_basic_dbeta_{file_endings}.csv", index=False)

dbeta threshold 

In [4]:
result_basic_dbeta = pd.read_csv(f"../result/result_basic_dbeta_{file_endings}.csv")

In [7]:
filter = result_basic_dbeta.loc[result_basic_dbeta["dbeta"].abs() > 0.03]
print(filter.shape)
filter

(306, 3)


,Unnamed: 0,dbeta,gene
50,cg10523679,-0.030600,ACADM
70,cg13642872,0.030652,ACSBG1
95,cg07568040,0.033123,ACVR1C
105,cg04892170,-0.040451,ADAM12
135,cg20395881,0.033628,ADARB2
...,...,...,...
7635,cg01174743,-0.032216,ZDHHC14
7642,cg00602811,0.039105,ZEB2
7676,cg14037413,-0.056093,ZNF143
7683,cg26649251,-0.042365,ZNF224


In [8]:
filter.to_csv(f"../result/result_basic_dbeta_{file_endings}_0.03.csv", index=False)